In [1]:
import cv2
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import csv
import time
import pickle

In [2]:
# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#CSV file reading
"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))

"""
Creates no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
def id_to_label(photo_ids):
    photo_labels = np.empty(shape=[0,9])
    for id in photo_ids:
        lst_labels = bus_to_labels_dict[photo_to_bus_dict[id]]
        ls = lst_labels[0].split(' ')
        labels = [0]*9
        for label in ls:
            if label !='':
                labels[int(label)] = 1
        labels = np.array(labels).reshape(1,-1)
        photo_labels = np.append(photo_labels,labels,axis=0)
    return photo_labels


GPU is active
Column names are photo_id, business_id
Processed 234843 lines.
234842
Column names are business_id, labels
Processed 2001 lines.
2000


In [3]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model

#inputpath = 'gen_data/vgg7avg/'
inputpath = 'gen_data/fv_1024/'
outpath = 'gen_data/vgg_fv_out_app8k_'

    
i = 1
file_list = sorted(os.listdir(inputpath))
train_list = file_list[:8000]
np_data = np.empty(shape=[0, 544])
np_label = np.empty(shape=[0,9])

for file_name in train_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['fv']
    #print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    
    #classifier_sgd.partial_fit(data, label, classes=2)
    np_data = np.append(np_data, data, axis = 0)
    np_label = np.append(np_label, label, axis = 0)
    
    #print(label, label.shape)
    #break
    i = i + 1
 
sio.savemat(outpath + 'data' + '.vgg', {'data':np_data}, do_compression=True)
sio.savemat(outpath + 'label' + '.vgg', {'label':np_label}, do_compression=True)

#classifier.fit(np_data, np_label)    
    

10.fv.mat - 1
100009.fv.mat - 2
100021.fv.mat - 3
100032.fv.mat - 4
100034.fv.mat - 5
100036.fv.mat - 6
100039.fv.mat - 7
10004.fv.mat - 8
100043.fv.mat - 9
100048.fv.mat - 10
100061.fv.mat - 11
100073.fv.mat - 12
100087.fv.mat - 13
100088.fv.mat - 14
10009.fv.mat - 15
100093.fv.mat - 16
100097.fv.mat - 17
1001.fv.mat - 18
100100.fv.mat - 19
100101.fv.mat - 20
100105.fv.mat - 21
100113.fv.mat - 22
100135.fv.mat - 23
10014.fv.mat - 24
100147.fv.mat - 25
10016.fv.mat - 26
100163.fv.mat - 27
100168.fv.mat - 28
100179.fv.mat - 29
100180.fv.mat - 30
100181.fv.mat - 31
100184.fv.mat - 32
100185.fv.mat - 33
1002.fv.mat - 34
100206.fv.mat - 35
100209.fv.mat - 36
10022.fv.mat - 37
100223.fv.mat - 38
10023.fv.mat - 39
100233.fv.mat - 40
100235.fv.mat - 41
100237.fv.mat - 42
100239.fv.mat - 43
100250.fv.mat - 44
100254.fv.mat - 45
100269.fv.mat - 46
100271.fv.mat - 47
100274.fv.mat - 48
100276.fv.mat - 49
100277.fv.mat - 50
100281.fv.mat - 51
100285.fv.mat - 52
100286.fv.mat - 53
100292.fv.mat - 

102853.fv.mat - 491
102858.fv.mat - 492
102863.fv.mat - 493
102870.fv.mat - 494
102871.fv.mat - 495
102883.fv.mat - 496
102886.fv.mat - 497
102888.fv.mat - 498
10289.fv.mat - 499
102896.fv.mat - 500
102904.fv.mat - 501
102905.fv.mat - 502
102913.fv.mat - 503
102914.fv.mat - 504
102921.fv.mat - 505
102927.fv.mat - 506
102928.fv.mat - 507
102930.fv.mat - 508
10294.fv.mat - 509
102940.fv.mat - 510
102941.fv.mat - 511
102944.fv.mat - 512
102948.fv.mat - 513
102950.fv.mat - 514
102960.fv.mat - 515
102983.fv.mat - 516
102991.fv.mat - 517
103005.fv.mat - 518
103010.fv.mat - 519
103014.fv.mat - 520
103026.fv.mat - 521
103028.fv.mat - 522
103030.fv.mat - 523
103032.fv.mat - 524
103034.fv.mat - 525
103045.fv.mat - 526
103054.fv.mat - 527
103065.fv.mat - 528
103067.fv.mat - 529
103068.fv.mat - 530
103073.fv.mat - 531
103080.fv.mat - 532
103089.fv.mat - 533
103091.fv.mat - 534
103099.fv.mat - 535
103120.fv.mat - 536
103124.fv.mat - 537
103126.fv.mat - 538
103129.fv.mat - 539
10313.fv.mat - 540
103

106140.fv.mat - 1083
106145.fv.mat - 1084
106146.fv.mat - 1085
106147.fv.mat - 1086
106148.fv.mat - 1087
106153.fv.mat - 1088
106164.fv.mat - 1089
106170.fv.mat - 1090
106171.fv.mat - 1091
106173.fv.mat - 1092
106185.fv.mat - 1093
106186.fv.mat - 1094
10620.fv.mat - 1095
106201.fv.mat - 1096
106211.fv.mat - 1097
106213.fv.mat - 1098
106214.fv.mat - 1099
106218.fv.mat - 1100
106227.fv.mat - 1101
106232.fv.mat - 1102
106237.fv.mat - 1103
106240.fv.mat - 1104
106247.fv.mat - 1105
106257.fv.mat - 1106
106270.fv.mat - 1107
106286.fv.mat - 1108
106289.fv.mat - 1109
106296.fv.mat - 1110
106307.fv.mat - 1111
10631.fv.mat - 1112
106311.fv.mat - 1113
10633.fv.mat - 1114
106338.fv.mat - 1115
106351.fv.mat - 1116
106358.fv.mat - 1117
106361.fv.mat - 1118
106366.fv.mat - 1119
106367.fv.mat - 1120
106380.fv.mat - 1121
106382.fv.mat - 1122
106386.fv.mat - 1123
1064.fv.mat - 1124
106437.fv.mat - 1125
10644.fv.mat - 1126
106444.fv.mat - 1127
106454.fv.mat - 1128
106458.fv.mat - 1129
106459.fv.mat - 113

108782.fv.mat - 1537
108788.fv.mat - 1538
108790.fv.mat - 1539
108792.fv.mat - 1540
108798.fv.mat - 1541
1088.fv.mat - 1542
108801.fv.mat - 1543
108804.fv.mat - 1544
108806.fv.mat - 1545
10881.fv.mat - 1546
108811.fv.mat - 1547
108813.fv.mat - 1548
108817.fv.mat - 1549
108819.fv.mat - 1550
108824.fv.mat - 1551
108826.fv.mat - 1552
108834.fv.mat - 1553
108835.fv.mat - 1554
108839.fv.mat - 1555
108843.fv.mat - 1556
108856.fv.mat - 1557
108857.fv.mat - 1558
10886.fv.mat - 1559
108862.fv.mat - 1560
108863.fv.mat - 1561
108867.fv.mat - 1562
108871.fv.mat - 1563
108872.fv.mat - 1564
108902.fv.mat - 1565
108903.fv.mat - 1566
108904.fv.mat - 1567
108906.fv.mat - 1568
108909.fv.mat - 1569
108911.fv.mat - 1570
108921.fv.mat - 1571
108930.fv.mat - 1572
108934.fv.mat - 1573
108956.fv.mat - 1574
108959.fv.mat - 1575
108963.fv.mat - 1576
108966.fv.mat - 1577
108969.fv.mat - 1578
108971.fv.mat - 1579
108976.fv.mat - 1580
108977.fv.mat - 1581
10899.fv.mat - 1582
108998.fv.mat - 1583
108999.fv.mat - 15

111314.fv.mat - 1984
111325.fv.mat - 1985
111333.fv.mat - 1986
111334.fv.mat - 1987
111336.fv.mat - 1988
111337.fv.mat - 1989
11134.fv.mat - 1990
111340.fv.mat - 1991
111341.fv.mat - 1992
111355.fv.mat - 1993
111358.fv.mat - 1994
111359.fv.mat - 1995
111360.fv.mat - 1996
111361.fv.mat - 1997
111368.fv.mat - 1998
111369.fv.mat - 1999
111370.fv.mat - 2000
111371.fv.mat - 2001
111373.fv.mat - 2002
111379.fv.mat - 2003
111403.fv.mat - 2004
111406.fv.mat - 2005
111415.fv.mat - 2006
111420.fv.mat - 2007
111436.fv.mat - 2008
111444.fv.mat - 2009
111445.fv.mat - 2010
11145.fv.mat - 2011
111452.fv.mat - 2012
111462.fv.mat - 2013
111463.fv.mat - 2014
111469.fv.mat - 2015
111470.fv.mat - 2016
111473.fv.mat - 2017
111486.fv.mat - 2018
111491.fv.mat - 2019
111494.fv.mat - 2020
111509.fv.mat - 2021
11151.fv.mat - 2022
111510.fv.mat - 2023
111518.fv.mat - 2024
111519.fv.mat - 2025
111533.fv.mat - 2026
111537.fv.mat - 2027
111538.fv.mat - 2028
11154.fv.mat - 2029
111542.fv.mat - 2030
111547.fv.mat - 2

...

134938.fv.mat - 6114
134940.fv.mat - 6115
134941.fv.mat - 6116
134948.fv.mat - 6117
134953.fv.mat - 6118
134955.fv.mat - 6119
134964.fv.mat - 6120
134966.fv.mat - 6121
134969.fv.mat - 6122
13497.fv.mat - 6123
134987.fv.mat - 6124
134988.fv.mat - 6125
134992.fv.mat - 6126
134994.fv.mat - 6127
134995.fv.mat - 6128
135006.fv.mat - 6129
135012.fv.mat - 6130
135036.fv.mat - 6131
135039.fv.mat - 6132
135045.fv.mat - 6133
135046.fv.mat - 6134
135057.fv.mat - 6135
135058.fv.mat - 6136
135064.fv.mat - 6137
135073.fv.mat - 6138
135081.fv.mat - 6139
135110.fv.mat - 6140
135112.fv.mat - 6141
135113.fv.mat - 6142
135115.fv.mat - 6143
135123.fv.mat - 6144
135125.fv.mat - 6145
135126.fv.mat - 6146
135127.fv.mat - 6147
135133.fv.mat - 6148
135141.fv.mat - 6149
135142.fv.mat - 6150
135153.fv.mat - 6151
135154.fv.mat - 6152
135164.fv.mat - 6153
135167.fv.mat - 6154
135168.fv.mat - 6155
135171.fv.mat - 6156
135178.fv.mat - 6157
135179.fv.mat - 6158
135180.fv.mat - 6159
135184.fv.mat - 6160
135191.fv.mat 

137432.fv.mat - 6533
137433.fv.mat - 6534
137439.fv.mat - 6535
137440.fv.mat - 6536
137444.fv.mat - 6537
137463.fv.mat - 6538
137465.fv.mat - 6539
137466.fv.mat - 6540
137474.fv.mat - 6541
137477.fv.mat - 6542
13749.fv.mat - 6543
137490.fv.mat - 6544
137492.fv.mat - 6545
137493.fv.mat - 6546
1375.fv.mat - 6547
137500.fv.mat - 6548
137501.fv.mat - 6549
137505.fv.mat - 6550
137510.fv.mat - 6551
137511.fv.mat - 6552
137516.fv.mat - 6553
137517.fv.mat - 6554
137525.fv.mat - 6555
137529.fv.mat - 6556
137546.fv.mat - 6557
137552.fv.mat - 6558
137557.fv.mat - 6559
13757.fv.mat - 6560
137580.fv.mat - 6561
137586.fv.mat - 6562
137598.fv.mat - 6563
137604.fv.mat - 6564
137619.fv.mat - 6565
13762.fv.mat - 6566
137624.fv.mat - 6567
137626.fv.mat - 6568
137637.fv.mat - 6569
137642.fv.mat - 6570
137648.fv.mat - 6571
137649.fv.mat - 6572
137651.fv.mat - 6573
137653.fv.mat - 6574
137683.fv.mat - 6575
137685.fv.mat - 6576
137689.fv.mat - 6577
13769.fv.mat - 6578
137690.fv.mat - 6579
137695.fv.mat - 658

139753.fv.mat - 6944
139770.fv.mat - 6945
13978.fv.mat - 6946
139789.fv.mat - 6947
139806.fv.mat - 6948
139808.fv.mat - 6949
139812.fv.mat - 6950
139817.fv.mat - 6951
139820.fv.mat - 6952
139822.fv.mat - 6953
139823.fv.mat - 6954
139827.fv.mat - 6955
139839.fv.mat - 6956
139845.fv.mat - 6957
139849.fv.mat - 6958
13985.fv.mat - 6959
139853.fv.mat - 6960
139857.fv.mat - 6961
139860.fv.mat - 6962
139870.fv.mat - 6963
139873.fv.mat - 6964
139878.fv.mat - 6965
139880.fv.mat - 6966
139889.fv.mat - 6967
139890.fv.mat - 6968
139898.fv.mat - 6969
1399.fv.mat - 6970
139900.fv.mat - 6971
139906.fv.mat - 6972
139907.fv.mat - 6973
13991.fv.mat - 6974
139912.fv.mat - 6975
139914.fv.mat - 6976
139919.fv.mat - 6977
13992.fv.mat - 6978
139925.fv.mat - 6979
139932.fv.mat - 6980
139933.fv.mat - 6981
139934.fv.mat - 6982
139937.fv.mat - 6983
139938.fv.mat - 6984
13994.fv.mat - 6985
139943.fv.mat - 6986
139944.fv.mat - 6987
139954.fv.mat - 6988
139955.fv.mat - 6989
139961.fv.mat - 6990
139963.fv.mat - 6991

142105.fv.mat - 7345
142107.fv.mat - 7346
142108.fv.mat - 7347
142111.fv.mat - 7348
142112.fv.mat - 7349
142115.fv.mat - 7350
142123.fv.mat - 7351
142125.fv.mat - 7352
142128.fv.mat - 7353
14213.fv.mat - 7354
142146.fv.mat - 7355
142147.fv.mat - 7356
14215.fv.mat - 7357
142153.fv.mat - 7358
142156.fv.mat - 7359
142166.fv.mat - 7360
142176.fv.mat - 7361
142178.fv.mat - 7362
142184.fv.mat - 7363
142185.fv.mat - 7364
142186.fv.mat - 7365
142187.fv.mat - 7366
142190.fv.mat - 7367
142196.fv.mat - 7368
142200.fv.mat - 7369
142205.fv.mat - 7370
142214.fv.mat - 7371
142218.fv.mat - 7372
142222.fv.mat - 7373
142235.fv.mat - 7374
142243.fv.mat - 7375
142245.fv.mat - 7376
142249.fv.mat - 7377
142252.fv.mat - 7378
142254.fv.mat - 7379
142259.fv.mat - 7380
142263.fv.mat - 7381
142272.fv.mat - 7382
142280.fv.mat - 7383
142293.fv.mat - 7384
14230.fv.mat - 7385
142305.fv.mat - 7386
142309.fv.mat - 7387
142310.fv.mat - 7388
142317.fv.mat - 7389
142320.fv.mat - 7390
14234.fv.mat - 7391
142341.fv.mat - 7

1444.fv.mat - 7753
144400.fv.mat - 7754
144402.fv.mat - 7755
14441.fv.mat - 7756
144421.fv.mat - 7757
144432.fv.mat - 7758
144437.fv.mat - 7759
144449.fv.mat - 7760
144483.fv.mat - 7761
144487.fv.mat - 7762
144492.fv.mat - 7763
144495.fv.mat - 7764
144508.fv.mat - 7765
144509.fv.mat - 7766
144512.fv.mat - 7767
144517.fv.mat - 7768
144533.fv.mat - 7769
144534.fv.mat - 7770
144538.fv.mat - 7771
144543.fv.mat - 7772
14455.fv.mat - 7773
144553.fv.mat - 7774
144555.fv.mat - 7775
144561.fv.mat - 7776
144571.fv.mat - 7777
144574.fv.mat - 7778
14458.fv.mat - 7779
144592.fv.mat - 7780
144595.fv.mat - 7781
144601.fv.mat - 7782
144605.fv.mat - 7783
144607.fv.mat - 7784
144613.fv.mat - 7785
144614.fv.mat - 7786
144618.fv.mat - 7787
144627.fv.mat - 7788
144649.fv.mat - 7789
144650.fv.mat - 7790
144651.fv.mat - 7791
144672.fv.mat - 7792
14468.fv.mat - 7793
144719.fv.mat - 7794
144721.fv.mat - 7795
144738.fv.mat - 7796
144753.fv.mat - 7797
144761.fv.mat - 7798
144765.fv.mat - 7799
144770.fv.mat - 780

In [10]:
"""
np_data = sio.loadmat('gen_data/vgg_fv_out_app8k_data.vgg')
np_data = np_data['data']
np_label = sio.loadmat('gen_data/vgg_fv_out_app8k_label.vgg')
np_label = np_label['label']
"""

random_state = np.random.RandomState(0)
classifier_2 = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, probability=True, random_state=random_state, 
                                        max_iter = 1000))

print("Training start")
t1 = time.time()
classifier_2.fit(np_data, np_label)
print("Training done")
print("Training time for 8000 Fisher vector processes images on 1vsRest took %0.3fs." % (time.time() - t1))


Training start


/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

Training done
Training time for 8000 Fisher vector processes images on 1vsRest took 738.207s.


/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [4]:
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.svm import LinearSVC
"""
np_data = sio.loadmat('gen_data/vgg_outputs_10k_data.vgg')
np_data = np_data['data']
np_label = sio.loadmat('gen_data/vgg_outputs_10k_label.vgg')
np_label = np_label['label']
"""

print("Training start")
t1 = time.time()
classifier_3 = MultiOutputClassifier(LinearSVC(random_state=0, max_iter=100))
classifier_3.fit(np_data, np_label)

print("Training done")
print("Training time of 8k fisher vector processed images on MultiOutputClassifier took %0.3fs." % (time.time() - t1))

Training start
Training done
Training time of 8k fisher vector processed images on MultiOutputClassifier took 0.708s.


In [9]:
### testing the model
#inputpath = 'gen_data/vgg7avg/'
inputpath = 'gen_data/fv_1024/'

print("Testing start")
t1 = time.time()
test_list = sorted(os.listdir(inputpath))
test_list = test_list[8000:]
#pred_list = []
#label_list = []
pred_list = np.empty(shape=[0,9])
label_list = np.empty(shape=[0,9])
i = 1
for file_name in test_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['fv']
    #print(data)
    #print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    pru = classifier_3.predict(data)
    i = i + 1
    pred_list = np.append(pred_list, pru, axis = 0)
    label_list = np.append(label_list, label, axis = 0)

print("Testing done")

print("Testing time of 2k fisher vector processed images on MultiOutputClassifier took %0.3fs." % (time.time() - t1))

Testing start
145957.fv.mat - 1
145967.fv.mat - 2
145969.fv.mat - 3
145970.fv.mat - 4
145971.fv.mat - 5
145972.fv.mat - 6
145974.fv.mat - 7
145976.fv.mat - 8
14598.fv.mat - 9
145980.fv.mat - 10
145986.fv.mat - 11
145988.fv.mat - 12
145991.fv.mat - 13
145992.fv.mat - 14
145998.fv.mat - 15
14600.fv.mat - 16
146000.fv.mat - 17
146004.fv.mat - 18
146005.fv.mat - 19
146008.fv.mat - 20
146009.fv.mat - 21
146011.fv.mat - 22
146018.fv.mat - 23
146031.fv.mat - 24
146042.fv.mat - 25
146045.fv.mat - 26
146049.fv.mat - 27
146055.fv.mat - 28
14606.fv.mat - 29
146061.fv.mat - 30
14607.fv.mat - 31
146074.fv.mat - 32
146083.fv.mat - 33
146087.fv.mat - 34
146097.fv.mat - 35
146099.fv.mat - 36
1461.fv.mat - 37
146101.fv.mat - 38
146111.fv.mat - 39
146112.fv.mat - 40
146119.fv.mat - 41
146120.fv.mat - 42
146129.fv.mat - 43
146134.fv.mat - 44
146142.fv.mat - 45
146144.fv.mat - 46
146152.fv.mat - 47
146157.fv.mat - 48
14617.fv.mat - 49
146170.fv.mat - 50
146175.fv.mat - 51
146179.fv.mat - 52
14618.fv.mat -

148156.fv.mat - 430
148158.fv.mat - 431
148160.fv.mat - 432
148162.fv.mat - 433
148165.fv.mat - 434
148169.fv.mat - 435
148188.fv.mat - 436
14819.fv.mat - 437
148203.fv.mat - 438
148204.fv.mat - 439
148211.fv.mat - 440
148212.fv.mat - 441
148213.fv.mat - 442
148224.fv.mat - 443
148230.fv.mat - 444
148234.fv.mat - 445
148244.fv.mat - 446
148250.fv.mat - 447
148267.fv.mat - 448
148269.fv.mat - 449
148274.fv.mat - 450
148277.fv.mat - 451
148282.fv.mat - 452
148289.fv.mat - 453
148302.fv.mat - 454
148304.fv.mat - 455
148308.fv.mat - 456
148313.fv.mat - 457
148323.fv.mat - 458
148330.fv.mat - 459
148333.fv.mat - 460
148340.fv.mat - 461
148343.fv.mat - 462
148346.fv.mat - 463
148349.fv.mat - 464
148358.fv.mat - 465
148360.fv.mat - 466
148366.fv.mat - 467
148373.fv.mat - 468
148374.fv.mat - 469
148376.fv.mat - 470
148378.fv.mat - 471
148380.fv.mat - 472
148382.fv.mat - 473
148385.fv.mat - 474
148390.fv.mat - 475
148405.fv.mat - 476
148430.fv.mat - 477
148432.fv.mat - 478
148436.fv.mat - 479
1

150606.fv.mat - 864
150610.fv.mat - 865
150624.fv.mat - 866
150627.fv.mat - 867
150629.fv.mat - 868
15063.fv.mat - 869
150630.fv.mat - 870
150631.fv.mat - 871
150634.fv.mat - 872
150635.fv.mat - 873
150636.fv.mat - 874
150639.fv.mat - 875
150640.fv.mat - 876
150641.fv.mat - 877
150644.fv.mat - 878
150653.fv.mat - 879
150654.fv.mat - 880
150657.fv.mat - 881
15067.fv.mat - 882
150678.fv.mat - 883
15069.fv.mat - 884
150695.fv.mat - 885
150700.fv.mat - 886
150729.fv.mat - 887
150736.fv.mat - 888
150741.fv.mat - 889
150747.fv.mat - 890
150754.fv.mat - 891
150768.fv.mat - 892
150773.fv.mat - 893
150776.fv.mat - 894
150782.fv.mat - 895
150798.fv.mat - 896
150807.fv.mat - 897
15082.fv.mat - 898
15083.fv.mat - 899
150830.fv.mat - 900
150837.fv.mat - 901
15084.fv.mat - 902
150847.fv.mat - 903
150849.fv.mat - 904
150866.fv.mat - 905
150869.fv.mat - 906
150873.fv.mat - 907
150875.fv.mat - 908
150876.fv.mat - 909
150877.fv.mat - 910
150882.fv.mat - 911
150883.fv.mat - 912
150886.fv.mat - 913
150894

152931.fv.mat - 1293
152938.fv.mat - 1294
152951.fv.mat - 1295
152954.fv.mat - 1296
152964.fv.mat - 1297
152969.fv.mat - 1298
152977.fv.mat - 1299
15298.fv.mat - 1300
152981.fv.mat - 1301
152996.fv.mat - 1302
153007.fv.mat - 1303
153015.fv.mat - 1304
153018.fv.mat - 1305
153024.fv.mat - 1306
153027.fv.mat - 1307
153032.fv.mat - 1308
153056.fv.mat - 1309
153060.fv.mat - 1310
153075.fv.mat - 1311
153079.fv.mat - 1312
153084.fv.mat - 1313
15310.fv.mat - 1314
153103.fv.mat - 1315
153106.fv.mat - 1316
153126.fv.mat - 1317
153136.fv.mat - 1318
153144.fv.mat - 1319
153145.fv.mat - 1320
153147.fv.mat - 1321
153148.fv.mat - 1322
153152.fv.mat - 1323
153160.fv.mat - 1324
153165.fv.mat - 1325
153167.fv.mat - 1326
153169.fv.mat - 1327
153170.fv.mat - 1328
153172.fv.mat - 1329
153195.fv.mat - 1330
153198.fv.mat - 1331
153217.fv.mat - 1332
153220.fv.mat - 1333
153223.fv.mat - 1334
153231.fv.mat - 1335
153236.fv.mat - 1336
153251.fv.mat - 1337
15326.fv.mat - 1338
15327.fv.mat - 1339
153287.fv.mat - 1

155415.fv.mat - 1735
155427.fv.mat - 1736
155433.fv.mat - 1737
155437.fv.mat - 1738
155440.fv.mat - 1739
155448.fv.mat - 1740
155452.fv.mat - 1741
155454.fv.mat - 1742
155469.fv.mat - 1743
155470.fv.mat - 1744
155473.fv.mat - 1745
155475.fv.mat - 1746
155485.fv.mat - 1747
155506.fv.mat - 1748
155508.fv.mat - 1749
155509.fv.mat - 1750
155517.fv.mat - 1751
15552.fv.mat - 1752
155531.fv.mat - 1753
155536.fv.mat - 1754
155537.fv.mat - 1755
155542.fv.mat - 1756
155557.fv.mat - 1757
155558.fv.mat - 1758
15558.fv.mat - 1759
155602.fv.mat - 1760
155604.fv.mat - 1761
155617.fv.mat - 1762
155620.fv.mat - 1763
155631.fv.mat - 1764
15564.fv.mat - 1765
155645.fv.mat - 1766
155655.fv.mat - 1767
155675.fv.mat - 1768
15568.fv.mat - 1769
155687.fv.mat - 1770
155688.fv.mat - 1771
155706.fv.mat - 1772
155708.fv.mat - 1773
15571.fv.mat - 1774
155715.fv.mat - 1775
155718.fv.mat - 1776
155725.fv.mat - 1777
155731.fv.mat - 1778
155735.fv.mat - 1779
155737.fv.mat - 1780
155739.fv.mat - 1781
155740.fv.mat - 17

In [8]:
#filename = 'vgg_fv_svc_multi_model.sav'
#pickle.dump(classifier_3, open(filename, 'wb'))


def verify_correctness(pred1, test_labels, threshold=0):
    cnt=0
    i = 1
    for x, y in zip(pred1, test_labels):
        x1= np.array(x)
        y1 = np.array(y)
        #print(x, y)
        #break
        c=0
        for a, b in zip(x, y):
            #print(a, b)
            #break
            if(a!=b): c+=1
        if c<=threshold:
            cnt+=1
        #if i > 50 and i < 100:
            #print(i,'-',cnt,'-', x1, y1)
        i = i + 1
    accuracy = cnt/len(test_list)
    return accuracy

#for p in pred_list[:50]:
#    print(p)
#print(label_list[0].shape)
#print(pred_list[1].shape, label_list[1].shape)
#pred_list = np.asarray(pred_list).reshape((-1, 9))
#label_list = np.asarray(label_list).reshape(-1, 9)

print(pred_list.shape, label_list.shape)

print("Accuracy score for strict match is %0.3f" % (verify_correctness(pred_list, label_list, 0)*100))
print("Accuracy score for maximum of 1 mistmatch is %0.3f" % (verify_correctness(pred_list, label_list, 1)*100))
print("Accuracy score for maximum of 2 mistmatch is %0.3f" % (verify_correctness(pred_list, label_list, 2)*100))

(5009, 9) (5009, 9)
Accuracy score for strict match is 51.764
Accuracy score for maximum of 1 mistmatch is 53.584
Accuracy score for maximum of 2 mistmatch is 59.285
